In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [29]:
roster = pd.read_csv('FIFA17final.csv',encoding = 'utf8')
defender_roster = roster[(roster['Position']=='LB')|(roster['Position']=='CB')|(roster['Position']=='RB')|(roster['Position']=='RWB')|(roster['Position']=='LWB')]
attacking_roster = roster[(roster['Position']=='CF')|(roster['Position']=='RW')|(roster['Position']=='LW')|(roster['Position']=='ST')]
midfield_roster = roster[ (roster['Position']=='CAM') | (roster['Position']=='CDM') | (roster['Position']=='LM') | (roster['Position']=='RM')|(roster['Position']=='CM')]
goalkeeper_roster = roster[ (roster['Position']=='GK')]

In [23]:
# seperate data and targe(ability)...remove all non number attributes
defender_roster_sk = defender_roster.drop('Surname',1)
defender_roster_sk = defender_roster_sk.drop('Position',1)
defender_roster_sk = defender_roster_sk.drop('Position.1',1)
defender_roster_sk = defender_roster_sk.drop('Name',1)
defender_roster_sk_target = defender_roster_sk['Ability']
defender_roster_sk_data = defender_roster_sk.drop('Ability',1)

In [32]:
def dropAll(roster,todrop):
    temp_roster = roster.copy()
    for name in todrop:
        temp_roster = temp_roster.drop(name,1)
    return temp_roster
def getAll(roster,tohave):
    temp_roster = roster.copy()
    return temp_roster[tohave]

In [34]:
tryDefender = dropAll(defender_roster,['Surname','Position','Position.1','Name','Ability'])
tryDefender = getAll(defender_roster,['Ability'])

array(['Ability'], dtype=object)

In [41]:
defender_data = np.array(defender_roster_sk_data)
defender_target = np.array(defender_roster_sk_target)

(5717, 33)


In [27]:
from sklearn import svm
clf = svm.LinearSVC()
clf.fit(defender_data,defender_target)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [35]:
clf.coef_

array([[-0.0008499 , -0.03021836, -0.01834279, ..., -0.03141874,
         0.13241409,  0.09615613],
       [-0.01030414, -0.05885596,  0.04384758, ...,  0.05014659,
        -0.05578016,  0.13050228],
       [-0.01621062, -0.0268339 ,  0.0058568 , ...,  0.0240213 ,
        -0.00206276, -0.03215261],
       ..., 
       [-0.04122211,  0.00404411,  0.01098317, ..., -0.01182795,
        -0.05463466, -0.00958628],
       [-0.03683668, -0.03033814, -0.00388758, ..., -0.021951  ,
        -0.05076825, -0.03166273],
       [-0.03031477,  0.0160945 , -0.0211666 , ...,  0.00701746,
         0.01395222, -0.03216353]])

In [53]:
test_mid_fit = dropAll(midfield_roster,['Surname','Position','Position.1','Name','Ability'])
# predict what the top 3 midfielders would be rated as defenders
clf.predict(np.array(test_mid_fit)[:3])

array([60, 49, 64])

In [55]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier()
knn.fit(defender_data,defender_target)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [91]:
#Use Knn to predict what the top 3 midfielders are rated as defenders
mid_knn_prediction = knn.predict(np.array(test_mid_fit))

In [92]:
# Lets see if anyone improved by switching to defense
mid_knn_pred_compare = np.array([mid_knn_prediction]).T > np.array(getAll(midfield_roster,['Ability']))

In [113]:
# names of players who improved
mid_names = np.array(getAll(midfield_roster,['Name']))
mid_prev_ability = np.array(getAll(midfield_roster,['Ability']))
better = mid_names[mid_knn_pred_compare.T[0]]
print(better)

IndexError: arrays used as indices must be of integer (or boolean) type